In [162]:
import os
import numpy as np
import pandas as pd
import torch
import torch as nn
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

df = pd.read_csv("https://raw.githubusercontent.com/mr-sarthakgupta/MUSIC-GENRE-CLASSIFICATION/main/music_train.csv")
dt = pd.read_csv("https://raw.githubusercontent.com/mr-sarthakgupta/MUSIC-GENRE-CLASSIFICATION/main/music_valid.csv")

train_dummies = pd.get_dummies(df['topic'])
valid_dummies = pd.get_dummies(dt['topic'])

df_z_scaled = df[['dating','violence','world/life','night/time','shake the audience','family/gospel','romantic','communication','obscene','music','movement/places','light/visual perceptions','family/spiritual','like/girls','sadness','feelings','danceability','loudness','acousticness','instrumentalness','valence','energy','age']].copy()

for column in df_z_scaled.columns:
	df_z_scaled[column] = (df_z_scaled[column] -
						df_z_scaled[column].mean()) / df_z_scaled[column].std()	

dt_z_scaled = dt[['dating','violence','world/life','night/time','shake the audience','family/gospel','romantic','communication','obscene','music','movement/places','light/visual perceptions','family/spiritual','like/girls','sadness','feelings','danceability','loudness','acousticness','instrumentalness','valence','energy','age']].copy()

for column in dt_z_scaled.columns:
	dt_z_scaled[column] = (dt_z_scaled[column] -
						dt_z_scaled[column].mean()) / dt_z_scaled[column].std()	

train_data = torch.tensor(pd.concat((df[['genre']], df_z_scaled, train_dummies), axis = 1).values)
valid_data = torch.tensor(pd.concat((dt[['genre']], dt_z_scaled, valid_dummies), axis = 1).values)

train_loader = DataLoader(train_data, batch_size=64, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size=64)


In [163]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.layers = nn.Sequential(
    nn.Linear(31, 20) ,
    nn.ReLU() ,
    nn.Linear(20, 15) ,
    nn.ReLU() ,
    nn.Linear(15, 10) ,
    nn.ReLU() ,
    nn.Linear(10, 8) ,
    #nn.ReLU() 
    #nn.Softmax()
    )
  #  self.fc1 = nn.Linear(31, 20)
  #   self.fc2 = nn.Linear(20, 15)
  #   self.fc3 = nn.Linear(15, 10)
  #   self.fc4 = nn.Linear(10, 8)

  def forward(self, x):
  #   x = F.relu(self.fc1(x))
  #   x = F.relu(self.fc2(x))
  #   x = F.relu(self.fc3(x))
  #   x = self.fc4(x)
  #   return x
    results = self.layers(x)
    return results
  
loss_fn = nn.CrossEntropyLoss()

model = NeuralNetwork()
print(model)

learning_rate = 1e-1
batch_size = 64
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=50, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
optimizer.zero_grad()
model.zero_grad()

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=31, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=15, bias=True)
    (3): ReLU()
    (4): Linear(in_features=15, out_features=10, bias=True)
    (5): ReLU()
    (6): Linear(in_features=10, out_features=8, bias=True)
  )
)


In [164]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, data in enumerate(dataloader):
        # Compute prediction and loss
        X=torch.tensor(data[:,1:], dtype=torch.float64)
        X=X.float()
        y=torch.tensor(data[:,0:1], dtype=torch.int64)
        y=y.squeeze(1)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #batch+=1
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [165]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 2.100148  [    0/ 4498]
Epoch 2
-------------------------------
loss: 1.924016  [    0/ 4498]
Epoch 3
-------------------------------
loss: 1.785041  [    0/ 4498]
Epoch 4
-------------------------------
loss: 1.763921  [    0/ 4498]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


Epoch 5
-------------------------------
loss: 1.862466  [    0/ 4498]
Epoch 6
-------------------------------
loss: 1.762955  [    0/ 4498]
Epoch 7
-------------------------------
loss: 1.709993  [    0/ 4498]
Epoch 8
-------------------------------
loss: 1.680042  [    0/ 4498]
Epoch 9
-------------------------------
loss: 1.809757  [    0/ 4498]
Epoch 10
-------------------------------
loss: 1.715756  [    0/ 4498]
Epoch 11
-------------------------------
loss: 1.669329  [    0/ 4498]
Epoch 12
-------------------------------
loss: 1.761013  [    0/ 4498]
Epoch 13
-------------------------------
loss: 1.759129  [    0/ 4498]
Epoch 14
-------------------------------
loss: 1.752358  [    0/ 4498]
Epoch 15
-------------------------------
loss: 1.678813  [    0/ 4498]
Epoch 16
-------------------------------
loss: 1.670821  [    0/ 4498]
Epoch 17
-------------------------------
loss: 1.531894  [    0/ 4498]
Epoch 18
-------------------------------
loss: 1.653616  [    0/ 4498]
Epoch 19
--

In [166]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, data in enumerate(dataloader):
            X=torch.tensor(data[:,1:], dtype=torch.float64)
            X=X.float()
            y=torch.tensor(data[:,0:1], dtype=torch.int64)
            y=y.squeeze(1)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [167]:
test_loop(valid_loader, model, loss_fn)

Test Error: 
 Accuracy: 40.9%, Avg loss: 0.023957 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


In [168]:
dv = pd.read_csv('https://raw.githubusercontent.com/mr-sarthakgupta/MUSIC-GENRE-CLASSIFICATION/main/music_test.csv')

test_dummies = pd.get_dummies(dv['topic'])

dv_z_scaled = dv[['dating','violence','world/life','night/time','shake the audience','family/gospel','romantic','communication','obscene','music','movement/places','light/visual perceptions','family/spiritual','like/girls','sadness','feelings','danceability','loudness','acousticness','instrumentalness','valence','energy','age']].copy()

dv_z_scaled = dv[['dating','violence','world/life','night/time','shake the audience','family/gospel','romantic','communication','obscene','music','movement/places','light/visual perceptions','family/spiritual','like/girls','sadness','feelings','danceability','loudness','acousticness','instrumentalness','valence','energy','age']].copy()

for column in dv_z_scaled.columns:
	dv_z_scaled[column] = (dv_z_scaled[column] -
						dv_z_scaled[column].mean()) / dv_z_scaled[column].std()	
      
test_data = torch.tensor(pd.concat(( dv_z_scaled, test_dummies), axis = 1).values)

test_loader = DataLoader(test_data, batch_size=1546)

with torch.no_grad():
        for batch, data in enumerate(test_loader):
            X=torch.tensor(data[:,0:], dtype=torch.float64)
            X=X.float()
            pred = model(X)
            result = pred.argmax(1)
            torch.set_printoptions(profile="full")
            print(result)
k = pd.DataFrame(result)
k.to_csv('result.csv',header=False, index=False)


tensor([4, 3, 2, 6, 3, 1, 1, 1, 4, 4, 1, 4, 4, 6, 5, 5, 3, 5, 3, 5, 4, 1, 0, 1,
        4, 5, 4, 1, 5, 4, 1, 6, 2, 1, 2, 1, 4, 4, 4, 5, 3, 1, 5, 1, 4, 5, 1, 1,
        1, 1, 6, 5, 4, 6, 6, 6, 2, 1, 1, 5, 1, 4, 0, 4, 4, 6, 0, 1, 3, 3, 1, 5,
        1, 1, 4, 0, 5, 3, 6, 3, 5, 3, 6, 1, 1, 3, 4, 2, 6, 1, 6, 4, 4, 4, 1, 1,
        6, 1, 4, 1, 0, 1, 1, 4, 1, 3, 1, 6, 5, 2, 1, 4, 1, 3, 6, 4, 4, 6, 4, 6,
        1, 1, 1, 0, 1, 4, 4, 6, 1, 0, 5, 1, 1, 5, 5, 4, 1, 4, 0, 1, 2, 6, 4, 6,
        6, 1, 1, 5, 5, 6, 5, 4, 3, 5, 1, 6, 0, 1, 1, 1, 1, 4, 4, 4, 1, 6, 0, 4,
        4, 5, 4, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 4, 6, 4, 5, 1, 6, 0, 6, 3,
        3, 4, 1, 4, 1, 1, 1, 4, 5, 6, 4, 4, 4, 1, 4, 1, 6, 5, 6, 4, 5, 5, 5, 3,
        3, 4, 1, 6, 3, 1, 5, 0, 5, 1, 4, 3, 1, 1, 5, 4, 1, 1, 6, 6, 3, 5, 0, 4,
        1, 4, 1, 1, 6, 3, 1, 1, 2, 1, 4, 3, 3, 0, 1, 0, 4, 3, 1, 6, 1, 3, 4, 0,
        0, 6, 1, 1, 1, 5, 3, 5, 1, 6, 3, 3, 4, 6, 6, 4, 1, 6, 3, 0, 2, 1, 1, 4,
        1, 1, 1, 5, 1, 4, 5, 1, 6, 1, 0,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
